<a href="https://colab.research.google.com/github/SergeyXiong/final-projects/blob/main/Ex5_1_XiongSuifu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [3]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [17]:
# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                    download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                    shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                    download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                    shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torchvision.models as models
from torchsummary import summary
model = models.resnet18()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False   

In [12]:
import torch.nn as nn
model_ft = models.resnet18(pretrained = True)
feature_extract = True
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, 10), nn.LogSoftmax(dim=1))
input_size= 224

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
model_ft = model_ft.to(device)

In [14]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
import torch.optim as optim
optimizer = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()

In [18]:
from datetime import datetime

def get_acc(output, label):
    """Calculation accuracy"""
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total


def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
   # """Model training"""
    prev_time = datetime.now()
    
    num_epochs = 5
    train_losses = []
    test_losses = []
    train_correct = []
    test_correct = []


    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        net = net.train()       # Train the model
        for im, label in train_data:
            im = im.to(device)          # (bs, 3, h, w)
            label = label.to(device)    # (bs, h, w)
            output = net(im)                    # Forward propagation
            loss = criterion(output, label)     # Loss function
            optimizer.zero_grad()               # Gradient zeroing
            loss.backward()                     # Backward propagation
            optimizer.step()                    # Gradient updates

            train_loss += loss.item()
            train_acc += get_acc(output, label)

        # Print run time
        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)

        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()        # Validate the model
            for im, label in valid_data:
                im = im.to(device)          # (bs, 3, h, w)
                label = label.to(device)    # (bs, h, w)
                output = net(im)                        # Forward propagation
                loss = criterion(output, label)         # Loss function
                valid_loss += loss.item()
                valid_acc += get_acc(output, label)
            # For each Epoch, print the result.
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, " %
                         (epoch, train_loss / len(train_data), train_acc / len(train_data), valid_loss / len(valid_data), valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data), train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)

train(model_ft, trainloader, testloader, 1, optimizer, criterion)

Epoch 0. Train Loss: 0.957982, Train Acc: 0.707385, Valid Loss: 0.700388, Valid Acc: 0.771855, Time 00:01:38
Epoch 1. Train Loss: 0.661609, Train Acc: 0.781841, Valid Loss: 0.635529, Valid Acc: 0.788667, Time 00:01:50
Epoch 2. Train Loss: 0.615014, Train Acc: 0.792979, Valid Loss: 0.607119, Valid Acc: 0.797172, Time 00:01:49
Epoch 3. Train Loss: 0.592944, Train Acc: 0.797634, Valid Loss: 0.595097, Valid Acc: 0.800336, Time 00:01:51
Epoch 4. Train Loss: 0.577832, Train Acc: 0.803149, Valid Loss: 0.589335, Valid Acc: 0.799150, Time 00:01:50


In [44]:
model2 = models.resnet50(pretrained = True)
set_parameter_requires_grad(model2, feature_extract)
num_ftrs2 = model2.fc.in_features
model2.fc = nn.Sequential(nn.Linear(num_ftrs2, 10), nn.LogSoftmax(dim=1))
#input_size= 224
#set_parameter_requires_grad(model2, feature_extract)


# Recreate the classifier layer and seed it to the target device
#model2.classifier = torch.nn.Sequential(
 #   torch.nn.Dropout(p=0.2, inplace=True), 
  #  torch.nn.Linear(in_features=1280, 
   #                 out_features=10, # same number of output units as our number of classes
    #                bias=True)).to(device)

In [45]:
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [47]:
#loss_fn = nn.CrossEntropyLoss()
#optimizer2 = torch.optim.Adam(model.parameters(), lr=0.001)
model2 = model2.to(device)
train(model2, trainloader, testloader, 5, optimizer, criterion)

Epoch 0. Train Loss: 2.351290, Train Acc: 0.105459, Valid Loss: 2.353993, Valid Acc: 0.102453, Time 00:02:53
Epoch 1. Train Loss: 2.351248, Train Acc: 0.105774, Valid Loss: 2.353061, Valid Acc: 0.101266, Time 00:03:24
Epoch 2. Train Loss: 2.351009, Train Acc: 0.104971, Valid Loss: 2.353308, Valid Acc: 0.099782, Time 00:03:23
Epoch 3. Train Loss: 2.351212, Train Acc: 0.104684, Valid Loss: 2.353841, Valid Acc: 0.099782, Time 00:03:23
Epoch 4. Train Loss: 2.351119, Train Acc: 0.105163, Valid Loss: 2.354239, Valid Acc: 0.100771, Time 00:03:23
